## Imports

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Flatten, MaxPooling1D
from tensorflow.keras.layers import Conv1D, Dense
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from kerastuner import HyperModel
from kerastuner.tuners import Hyperband
from kerastuner.tuners import RandomSearch
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import time

### Enables Tensorflow to use the GPU

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Paths

In [4]:
paths = {"pickels": "../../assets/audio_sentiment_data_v2/pickles",
         "models": "../../assets/audio_sentiment_data_v2/models",
         "tuners": "../../assets/audio_sentiment_data_v2/tuners"}

## Loading in the data splits from the pickels

In [5]:
pickle_in = open(f"{paths['pickels']}/X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open(f"{paths['pickels']}/X_val.pickle","rb")
X_val = pickle.load(pickle_in)

pickle_in = open(f"{paths['pickels']}/X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open(f"{paths['pickels']}/y_train.pickle","rb")
y_train = pickle.load(pickle_in)

pickle_in = open(f"{paths['pickels']}/y_val.pickle","rb")
y_val = pickle.load(pickle_in)

pickle_in = open(f"{paths['pickels']}/y_test.pickle","rb")
y_test = pickle.load(pickle_in)

## Reshaping the input data

In [6]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

print(X_train.shape, X_val.shape, X_test.shape)

(2445, 37, 1) (489, 37, 1) (326, 37, 1)


## Defining the Model using HyperModel Class to perform hyperparameters tuning

In [6]:
class CNNHyperModel(HyperModel):
    
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes
        
    def build(self, hp):
        model = Sequential()
        
        model.add(
            Conv1D(
                filters=hp.Choice(
                    'num_filters_1',
                    values=[16, 32, 64],
                    default=64,
                ),
                kernel_size=3,
                activation=hp.Choice(
                    'conv_activation_1',
                    values=['relu', 'tanh'],
                    default='relu'
                ),
                input_shape=self.input_shape
            )
        )
        
        model.add(
            Conv1D(
                filters=hp.Choice(
                    'num_filters_2',
                    values=[16, 32, 64],
                    default=64,
                ),
                kernel_size=3,
                activation=hp.Choice(
                    'conv_activation_2',
                    values=['relu', 'tanh'],
                    default='relu'
                )
            )
        )
        
        model.add(MaxPooling1D(pool_size=2))
        
        model.add(
            Dropout(rate=hp.Float(
                'dropout_1',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            ))
        )
        
        model.add(Flatten())
        
        model.add(
            Dense(
                units=hp.Int(
                    'dense_units',
                    min_value=16,
                    max_value=72,
                    step=16,
                    default=32
                ),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        
        model.add(
            Dropout(rate=hp.Float(
                'dropout_2',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            ))
        )
        
        model.add(Dense(self.num_classes, activation='softmax'))
        
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model       

## Creating an instance of the HyperModel Class

In [7]:
NUM_CLASSES = 5
INPUT_SHAPE = (37, 1)

hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

## Defining the Hyperband Tuner

In [ ]:
SEED = 1

hb_tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=40,
    hyperband_iterations=2,
    executions_per_trial=2,
    seed=SEED,
    directory=f'{paths["tuners"]}/hyperband_tuner',
    project_name='AudioSentimentClassifier'
)

### Searching for the best model

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

hb_tuner.search(X_train,
             y_train,
             validation_data=(X_val, y_val),
             epochs=40,
             callbacks=[stop_early])

### Retrieving the best model

In [ ]:
best_model = hb_tuner.get_best_models(num_models=1)[0]
best_hps = hb_tuner.get_best_hyperparameters(num_trials=1)[0]
best_model.summary()

In [ ]:
model = hb_tuner.hypermodel.build(best_hps)

history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), verbose=0)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

### Retraining with optimal epochs

In [ ]:
hypermodel = hb_tuner.hypermodel.build(best_hps)

# Retrain the model
history = hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_data=(X_val, y_val), verbose=0)
print(f"val_acc = {history.history['val_accuracy']}\nval_loss = {history.history['val_loss']}")

### Saving the tuned model

In [ ]:
name = f"hyperband_tuned_best_model_val_acc{history.history['val_accuracy']}_val_loss{history.history['val_loss']}.hdf5"
filepath = f"{paths['models']}/{name}.hdf5"
hypermodel.save(filepath)

## Defining the RandomSearch Tuner

In [8]:
SEED = 1

rs_tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    executions_per_trial=2,
    directory=f'{paths["tuners"]}/random_search_tuner',
    project_name='AudioSentimentClassifier')

### Searching for the best model

In [9]:
rs_tuner.search(X_train,
                y_train,
                validation_data=(X_val, y_val),
                epochs=30)

Trial 20 Complete [00h 00m 33s]
val_accuracy: 0.7770960927009583

Best val_accuracy So Far: 0.8333333134651184
Total elapsed time: 00h 10m 46s
INFO:tensorflow:Oracle triggered exit


### Retrieving the best model

In [10]:
best_model = rs_tuner.get_best_models(num_models=1)[0]
best_hps = rs_tuner.get_best_hyperparameters(num_trials=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 35, 64)            256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 33, 64)            12352     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 16, 64)            0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 64)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                65600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0

In [14]:
best_hps.get('conv_activation_2')

'tanh'

In [15]:
model = rs_tuner.hypermodel.build(best_hps)

history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), verbose=0)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 31


### Retraining with optimal epochs

In [ ]:
hypermodel = rs_tuner.hypermodel.build(best_hps)

# Retrain the model
history = hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_data=(X_val, y_val), verbose=0)

In [19]:
val_acc = history.history['val_accuracy']
max_val_acc = max(val_acc)
max_val_acc_loss = history.history['val_loss'][val_acc.index(max(val_acc))]
print(f"val_acc = {max_val_acc}\nval_loss = {max_val_acc_loss}")

val_acc = 0.8179959058761597
val_loss = 0.4643225073814392


### Saving the tuned model

In [21]:
name = f"random_search_tuned_best_model_val_acc{max_val_acc}_val_loss{max_val_acc_loss}.hdf5"
filepath = f"{paths['models']}/{name}.hdf5"
hypermodel.save(filepath)

In [22]:
eval_result = hypermodel.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

11/11 [==============================] - 0s 6ms/step - loss: 0.4291 - accuracy: 0.8436
[test loss, test accuracy]: [0.4290520250797272, 0.8435583114624023]


In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.close()

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.close()

In [ ]:
predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis=1)

new_y_test = np.argmax(y_test, axis=1)

print(classification_report(new_y_test, predictions))

In [ ]:
pickle_in = open("labels.pickle","rb")
lb= pickle.load(pickle_in)
labels = lb.classes_

In [ ]:
%matplotlib inline
conf_matrix = pd.DataFrame(tf.math.confusion_matrix(new_y_test, predictions).numpy(), index=labels, columns=labels)
conf_matrix.index.name = 'Actual'
conf_matrix.columns.name = 'Predicted'
fig_dims = (8, 6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.heatmap(conf_matrix, annot=True, cmap="Blues", annot_kws={"size": 16}, fmt='d')